<a href="https://colab.research.google.com/github/osmarbraz/exemplos_Llama/blob/main/Exemplos_Chunk_Overlap_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Exemplo de manipulação de texto usando Langchain

Cria a divisão de um texto em pedaços(chunks) para submeter a um LLM. Realiza a sobreposição(overlap) de parte do texto para manter a semântica entre os chunks.

Semelhante a uma janela deslizante.

Teste online da chunk(divisão) e overlap(overlap) usando um arquivo texto: https://chunkerizer.streamlit.app/

Entendendo o chunk e overlap
https://gustavo-espindola.medium.com/chunk-division-and-overlap-understanding-the-process-ade7eae1b2bd

LangChain: como dividir corretamente seus chunks(pedaços):
https://www.youtube.com/watch?v=n0uPzvGTFI0

Estratégias de chucking: https://www.pinecone.io/learn/chunking-strategies/

Como agrupar dados de texto - uma análise comparativa: https://towardsdatascience.com/how-to-chunk-text-data-a-comparative-analysis-3858c4a0997a

# 0 - Preparação do ambiente
Preparação do ambiente para execução do exemplo.

## Tratamento de logs

Método para tratamento dos logs.

In [ ]:
# Biblioteca de logging
import logging

# Formatando a mensagem de logging
logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)

## Identificando o ambiente Colab

Cria uma variável para identificar que o notebook está sendo executado no Google Colaboratory.

In [ ]:
# Se estiver executando no Google Colaboratory
import sys

# Retorna true ou false se estiver no Google Colaboratory
IN_COLAB = "google.colab" in sys.modules

## Funções auxiliares

Função auxiliar para formatar o tempo como `hh: mm: ss`

In [ ]:
# Import das bibliotecas.
import time
import datetime

def formataTempo(tempo):
    """
      Pega a tempo em segundos e retorna uma string hh:mm:ss
    """
    # Arredonda para o segundo mais próximo.
    tempo_arredondado = int(round((tempo)))

    # Formata como hh:mm:ss
    return str(datetime.timedelta(seconds=tempo_arredondado))

# 1 - Instalação das bibliotecas

Manipulação de textos para LLM.

In [ ]:
!pip install langchain==0.0.320

Biblioteca para manipular pdf

In [ ]:
!pip install pypdf==3.16.4

# 2 - Manipulando letras

https://medium.com/@onkarmishra/using-langchain-for-question-answering-on-own-data-3af0a82789ed

In [ ]:
#Import de bibliotecas
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

chunk_size =26
chunk_overlap = 4

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
# Divisor recursivo de texto
text1 = 'abcdefghijklmnopqrstuvwxyz'
r_splitter.split_text(text1)
# Saída - ['abcdefghijklmnopqrstuvwxyz']

['abcdefghijklmnopqrstuvwxyz']

In [ ]:

# Dividor de caracter texto
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
r_splitter.split_text(text2)
# Saída - ['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

['abcdefghijklmnopqrstuvwxyz', 'wxyzabcdefg']

In [ ]:
# Recursive text Splitter
# Dividor recursivo de texto
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [ ]:
# Dividor de caracter texto
c_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m n o p q r s t u v w x y z']

['a b c d e f g h i j k l m n o p q r s t u v w x y z']

In [ ]:
# Character Text Splitter with separator defined
# Divisor de caracter texto com separador definido
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)
# Saída - ['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

# 3 - Manipulando documentos Texto

## Cria o texto

In [ ]:
# trecho de https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116979', #As Vítimas algozes


texto = """
No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode conhecer por suas feições e modos
característicos, se nos é lícito dizer assim: uma se parece com todas e não há
hipótese em que alguma delas, por mais dissimulada que seja, chegue a perder o
cará­ter da família.
É uma pequena casa de taipa e coberta de telha, tendo às
vezes na frente varanda aberta pelos três lados, também coberta de telha e com
o teto sustido por esteios fortes, mas rudes e ainda mesmo tortos; as paredes
nem sempre são caiadas, o chão não tem assoalho nem ladrilho; quando há
varanda, abrem-se para ela uma porta e uma janela; dentro está a ven­da: entre
a porta e a janela encostado à parede um banco de pau, defronte um balcão tosco
e no bojo ou no espaço que se vê além, grotesca armação de tábuas contendo
garrafas, botijas, latas de tabaco em pó, a um canto algumas voltas de fumo em
rolo e uma ruim manta de carne-seca. Eis a venda.
Há muitas que nem chegam à opulência da que aí fica
descrita; em todas porém aparece humilde no fundo do quase vazio bojo a porta
baixa que comunica pelo corredor imundo com dois ou mais quartos escuros, onde
se recolhem as pingues colheitas agrícolas do vendelhão que aliás não tem
lavoura.
"""

## Carrega e divide o texto

In [ ]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.create_documents([texto])

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos texto em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos texto em 0.0008533000946044922 segundos!


In [ ]:
print(len(chunks))

5


In [ ]:
print(chunks)

[Document(page_content='No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase\nsempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,\né a inimiga hipócrita que rende vassalagem à sua vítima.\nA venda de que falo é uma taberna especialíssima\nque não poderia existir, manter-se, medrar em outras condições locais, e em\noutras condições do trabalho rural, e nem se confunde com a taberna regular que', metadata={'start_index': 1}), Document(page_content='outras condições do trabalho rural, e nem se confunde com a taberna regular que\nem toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,\nonde os lavradores ricos e pobres se provêem do que precisa a casa, quando não\nlhes é possível esperar pelas remessas dos seus consigna\xadtários ou fregueses.\nEssa parasita das fazendas e estabelecimentos agrícolas\ndas vizinhanças facilmente se pode conhecer por suas feições e modos', metadata={'start_index': 353}

## Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [ ]:
for i, chunk in enumerate(chunks):
    print('chunk #',i,' tamanho :', len(chunk.page_content),' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  tamanho : 431  start_index: 1

No interior e principalmente longe da vila, ou da freguesia e dos povoados há quase
sempre uma venda perto da fazenda: é a parasita que se apega à árvore; pior que isso,
é a inimiga hipócrita que rende vassalagem à sua vítima.
A venda de que falo é uma taberna especialíssima
que não poderia existir, manter-se, medrar em outras condições locais, e em
outras condições do trabalho rural, e nem se confunde com a taberna regular que
-----------------------------------------------------------------------
chunk # 1  tamanho : 444  start_index: 353

outras condições do trabalho rural, e nem se confunde com a taberna regular que
em toda parte se encontra, quanto mais com as casas de grande ou pequeno comércio,
onde os lavradores ricos e pobres se provêem do que precisa a casa, quando não
lhes é possível esperar pelas remessas dos seus consigna­tários ou fregueses.
Essa parasita das fazendas e estabelecimentos agrícolas
das vizinhanças facilmente se pode

# 4 - Manipulando Documentos PDF

## Download dos pdfs

In [ ]:
# Import das bibliotecas
import subprocess

url1 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=116977'
destino1 = 'arquivo1.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino1])

url2 = 'https://www.literaturabrasileira.ufsc.br/documentos/?action=download&id=92390'
destino1 = 'arquivo2.pdf'

# Executa o comando wget no prompt
subprocess.call(["wget", url1, "-O", destino1])

0

## Carrega os pdfs

In [ ]:
# Import das bibliotecas
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

documentos = []

# Define o diretório
diretorio = '/content'

 # Carrega os documentos
documentos = PyPDFDirectoryLoader(diretorio).load()

## Carrega e divide os pdfs

In [ ]:
# Import das bibliotecas
import os
import time
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Parâmetros
chunk_tamanho = 500
chunk_sobreposicao = 100

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_tamanho,
    chunk_overlap  = chunk_sobreposicao, # Número de tokens sobrepostos entre chunks(pedaços)
    length_function = len, # Usa o comprimento do texto como medida de tamanho
    add_start_index = True, #
)

# Guarda o tempo de início
tempo_inicio = time.time()

# Calcula os chunks dos textos
chunks = text_splitter.split_documents(documentos)

tempo_final = time.time()

print(f"Carregando e dividindo {len(texto)} documentos pdfs em {tempo_final - tempo_inicio} segundos!")

Carregando e dividindo 1918 documentos pdfs em 0.06731224060058594 segundos!


## Mostra os chunks

A sobreposição se encontra ao final e início de cada pedaço(chunk). A variável 'start_index' define onde começa o texto sem a sobreposição.

In [ ]:
for i, chunk in enumerate(chunks):
  if i < 20:
    print('chunk #',i,' tamanho :', len(chunk.page_content),' start_index:', chunk.metadata.get('start_index') )
    print()
    print(chunk.page_content)
    print('-----------------------------------------------------------------------')

chunk # 0  tamanho : 375  start_index: 0

J MAS-ALGOZE 
ROMAM 
~% 
LIVRARIA DE B. L. GARNIER 
«9, rua do Ouvidor, CO 
Grandesortimento del.ivros clássicos, Medicina, 
Sciencias e Arlís, Junsprudcncia, Littsratura, 
Novellas, lllustrações, Educação, Devoção, Atlas, 
Happas geographicos, etc, etc. 
Livros francezes, portuguezes, inqlezes, italianos, ele-, 
Encarrega-se áe qualquer conmissio de Liuros-
BIO DE JIMIIIO
-----------------------------------------------------------------------
chunk # 1  tamanho : 78  start_index: 0

Ie ne fay rien 
sans 
Gayeté 
(Montaigne, Des livres) 
Ex Libris 
José Mindlin
-----------------------------------------------------------------------
chunk # 2  tamanho : 19  start_index: 0

AS VICTIMAS-ALGOZES
-----------------------------------------------------------------------
chunk # 3  tamanho : 154  start_index: 0

AS VICTIMAS-ALGOZES 
QUADROS DA ESCRAVIDÃO 
ROMANCES 
POR 
JOAQUIM MANOEL DE MACEDO 
TOMO I 
fyo DE JANEIRO 
Typ. AMERICANA , roa dos Ourives n